<a href="https://colab.research.google.com/github/hihi5456/pytorch/blob/main/Meta_Pseudo_Label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm

     |████████████████████████████████| 296kB 6.1MB/s 


In [2]:
import torchvision
from torchvision import transforms, datasets

import torch.utils.data.dataloader
import matplotlib.pyplot as plt


import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,ConcatDataset

import numpy  as np

from sklearn.metrics import roc_auc_score, auc
import timm

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(456)
if device =='cuda':
    torch.cuda.manual_seed_all(456)

In [4]:
model_T=timm.models.efficientnet.efficientnet_b0(pretrained=False)


model_S_t=timm.models.efficientnet.efficientnet_b0(pretrained=False)
#model_S_tp

In [5]:
criterion=nn.CrossEntropyLoss()


In [47]:
optimizer_T=optim.RMSprop(model_T.parameters(), lr=0.001, momentum=0.9)

lr_S=0.001
optimizer_S=optim.RMSprop(model_S_t.parameters(), lr=lr_S, momentum=0.9)


## **Update Student**

In [7]:
def Update_student(model_S_t,PL_data, L_data):
  #Student model update
  PL_input, PL_label=PL_data
  PL_input=PL_input.to(device)
  PL_label=PL_label.to(device)

  optimizer_S.zero_grad()

  output=model_S_t(PL_data)
  loss=criterion(PL_label, output)
  loss.backward()
  optimizer_S.step()
  model_S_tp=model_S_t

  #Get gradient from updated student model  
  L_input, L_label=L_data
  L_input=L_input.to(device)
  L_label=L_label.to(device)
  
  output=model_S_tp(L_input)
  loss=criterion(L_label,output)
  loss.backward()
  
  return get_h(model_S_t,model_S_tp)

## **Update Teacher**

In [8]:
def Update_teacher(model_T, PL_data, L_data, h):
 
  PL_input, PL_label=PL_data
  PL_input=PL_input.to(device)
  PL_label=PL_label.to(device)
  
  L_input, L_label=L_data
  L_input=L_input.to(device)
  L_label=L_label.to(device)


  optimizer_T.zero_grad()
  
  with torch.no_grad():
    model_T.eval()
    UDA_label=model_T(L_input)
    
  model_T.train()

  loss=h*criterion(PL_label,model_T(PL_input)) + criterion( L_label,model_T(L_input)) \
      + criterion(UDA_label, model_T(RandAug(L_input)))
  
  loss.backward()
  optimizer_T.step()


## **Get Feedback coefficient**(just for EfficientNet B0)

In [ ]:
#Just for EfficientNet B0
def get_h(model_S_t,model_S_tp,lr_S):
  
  h=0



  codes='''
  if x != None:
    temp=x*y
    h=h+temp.sum()*lr_S
  '''

  #model_S_t.state_dict().keys()
  for keys in model_S_t.state_dict().keys():
    print(keys)
    if keys[2]!='o':
      
      code='x=model_S_t.{}.grad'.format(keys)
      exec(code)
      code='y=model_S_tp.{}.grad'.format(keys)
      exec(code)
      exec(codes)

      
    else:
    
      modulename=keys[11:]
      keys=list(keys)
    
      keys[6]='['
      keys[8]=']'
      keys[10]=keys[9]
      keys[9]='['
      keys[11]=']'
      keys=''.join(keys)
      code='x=model_S_t.{}.{}.grad'.format(keys[:12],modulename)
      exec(code)
      code='y=model_S_tp.{}.{}.grad'.format(keys[:12],modulename)
      exec(code)
      exec(codes)

  return h

# ***Model gradient check***

In [ ]:
# just for efficientnet b0
codes='''
if x != None:
  print(x.shape)
'''

model_S_t.state_dict().keys()
for keys in model_S_t.state_dict().keys():
  print(keys)
  if keys[2]!='o':
    code='x=model_S_t.{}.grad'.format(keys)
    exec(code)
    exec(codes)

    
  else:
    
    modulename=keys[11:]
    keys=list(keys)
    
    keys[6]='['
    keys[8]=']'
    keys[10]=keys[9]
    keys[9]='['
    keys[11]=']'
    keys=''.join(keys)
    code='x=model_S_t.{}.{}.grad'.format(keys[:12],modulename)
    exec(code)
    exec(codes)

## **Data loader**

## **Training**